In [2]:
import pandas as pd
import os
import polars as pl
import matplotlib.pyplot as plt
import torch
from torch import nn
import kaggle_evaluation.jane_street_inference_server
from sklearn.linear_model import LinearRegression

Load in features and responders metadata

In [36]:
DIR = "./jane-street-real-time-market-data-forecasting"

features = pl.read_csv(DIR + "/features.csv")
responders = pl.read_csv(DIR + "/responders.csv")

Load in test data

In [37]:
test = pl.read_parquet(DIR + "/test.parquet/date_id=0/part-0.parquet")
test

row_id,date_id,time_id,symbol_id,weight,is_scored,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,…,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
i64,i16,i16,i8,f32,bool,f32,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
0,0,0,0,3.169998,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
1,0,0,1,2.165993,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
2,0,0,2,3.06555,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,-0.0,-0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
3,0,0,3,2.698642,true,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,-0.0,-0.0
4,0,0,4,1.80333,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,-0.0,null,-0.0,0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
34,0,0,34,3.240565,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,null,-0.0,null,-0.0,-0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,null,-0.0,null,null,-0.0,null,-0.0,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,null,null,0.0,0.0,0.0,0.0
35,0,0,35,1.057221,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,…,null,0.0,null,-0.0,0.0,0.0,0.0,0.0,null,0.0,null,null,0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,-0.0,-0.0
36,0,0,36,0.907022,true,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,null,-0.0,null,-0.0,-0.0,-0.0,null,-0.0,-0.0,0.0,-0.0,null,null,-0.0,-0.0,-0.0,…,null,-0.0,null,-0.0,0.0,-0.0,0.0,0.0,null,0.0,null,null,0.0,null,-0.0,0.0,null,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0,-0.0,null,null,0.0,0.0,0.0,0.0


In [38]:
def read_in_data():
    base_str = "/train.parquet"
    filename = "/part-0.parquet"

    data_by_id = {}
    dropped_nulls = {}
    
    #read in all the data
    for i in range(10):
        partition = "/partition_id=" + str(i)
        read = pl.read_parquet(DIR + base_str + partition + filename)
        read = read[:len(read) // 2]
        ids = read['symbol_id'].unique()

        for i in range(len(ids)):
            symbol_id = int(ids[i])
            filtered = read.filter(pl.col("symbol_id") == symbol_id)
            
            #drop weight column since it only influences scoring
            filtered.drop_in_place('weight')
            
            if symbol_id not in data_by_id:
                data_by_id[symbol_id] = filtered
            else:
                data_by_id[symbol_id].extend(filtered)

    for symbol_id in data_by_id.keys():
        symbol_id = int(symbol_id)

        if symbol_id not in dropped_nulls:
            dropped_nulls[symbol_id] = data_by_id[symbol_id].drop_nulls()
        else:
            dropped_nulls[symbol_id].extend(data_by_id[symbol_id].drop_nulls())
        
    agg_data = {}
    
    for symbol_id in dropped_nulls.keys():
        symbol_id = int(symbol_id)
        new_df = pl.DataFrame()
        curr_df = dropped_nulls[symbol_id]
        time_ids = curr_df['date_id'].unique()

        for j in range(len(time_ids)):
            curr_time_id = time_ids[j]
            same_time = curr_df.filter(pl.col('date_id') == curr_time_id)
            total = len(same_time)
            
            if new_df.is_empty():
                new_df = pl.DataFrame(same_time.sum() / total)
            else:
                new_df.extend(same_time.sum() / total)
    
        new_df.drop_in_place('time_id')
        agg_data[symbol_id] = new_df

    agg_training_data = pl.DataFrame()

    for i in range(len(ids)):
        symbol_id = ids[i]
        
        if agg_training_data.is_empty():
            agg_training_data = pl.concat([agg_training_data, agg_data[symbol_id]])
        
        else:
            agg_training_data.extend(agg_data[symbol_id])
    
    return agg_training_data, agg_training_data['symbol_id'].unique()

train, ids = read_in_data()

In [ ]:
ids = ids.cast(pl.Int8)

date_id,symbol_id,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,…,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
553.0,0.0,2.013941,1.292343,1.81205,1.849068,0.440201,0.948113,0.122665,0.4326,-2.127602,11.0,7.0,76.0,1.912674,0.706286,1.241098,-0.315865,-0.197647,-0.276523,1.029011,-1.089639,2.318863,-0.148865,0.595681,0.824761,0.621569,0.37841,1.108164,1.495799,1.130262,0.022011,-0.17891,-0.32181,2.573313,-0.424322,2.831493,…,0.455364,0.599965,-1.071355,-0.443253,-0.68296,-0.204628,-2.378987,-0.940073,-0.158733,-0.376406,-1.592721,-0.172186,-0.078757,-0.124456,0.996328,-0.995877,1.859685,0.611992,1.169616,1.73828,0.571538,1.171094,-0.236857,-0.505843,-0.165729,-0.38803,-0.192449,-0.429288,-0.075958,0.036015,-0.03608,0.490821,1.242675,0.156754,0.703481,1.356552,0.290338
554.0,0.0,1.308025,0.165198,1.290724,1.267263,-0.214182,-0.34573,-0.073516,-0.18006,0.511139,11.0,7.0,76.0,0.480354,0.141877,0.275949,-0.34648,-0.278102,-0.36054,1.031542,-1.008696,2.247149,-0.296195,0.4439,0.684728,1.037862,1.370785,1.043949,1.576174,0.869166,-0.050006,-0.331859,-0.148651,2.331386,-0.724733,2.642955,…,0.14347,0.172195,-0.503926,-0.15869,-0.176066,-0.654094,-1.81148,-0.614141,-0.066149,-0.210956,-1.43251,-0.313611,-0.230842,-0.274072,1.019645,-1.188255,0.431414,0.106297,0.226803,0.489445,0.131714,0.282938,-0.286459,-0.482672,-0.200277,-0.37504,-0.229373,-0.413216,-0.009159,-0.256007,-0.001491,-0.023604,-0.169132,0.008305,-0.006774,-0.155766,0.033297
555.0,0.0,1.258477,-0.076692,1.285174,1.292239,-0.027652,0.011702,-0.032355,-0.117747,-0.329927,11.0,7.0,76.0,0.095979,-0.05203,-0.043304,-0.375075,-0.32719,-0.394974,-0.590368,0.509876,1.947477,-0.134973,0.797803,0.470683,1.54787,1.530932,0.837595,1.704533,1.162891,-0.104783,-0.419231,-0.139533,2.432508,0.626563,2.530757,…,0.15733,0.340143,-0.463043,-0.142314,-0.293874,0.716893,-0.170615,0.285231,0.01056,0.089889,-0.972459,-0.270933,-0.235193,-0.283959,0.248875,0.473726,0.088685,-0.032585,-0.021892,0.11108,-0.056707,-0.046041,-0.273585,-0.193488,-0.200692,-0.135081,-0.231391,-0.160426,0.010026,0.175302,0.014204,-0.148629,-0.395728,-0.050435,-0.173552,-0.375334,-0.076376
556.0,0.0,1.300598,-0.435179,1.344048,1.360433,-0.079506,-0.154848,-0.040517,-0.07214,0.096866,11.0,7.0,76.0,-0.405203,-0.19701,-0.314084,-0.420815,-0.382657,-0.453053,0.660968,0.336354,1.771308,-0.138153,0.971693,0.485427,1.569678,2.036034,0.48541,2.13282,0.599653,-0.097419,-0.392113,-0.228418,2.336106,0.380331,2.453971,…,0.074362,0.149768,-0.015907,-0.020577,0.038686,-0.197894,-0.207483,0.058592,0.033355,0.114532,-1.34787,-0.31822,-0.255906,-0.289804,0.611474,0.279397,-0.346319,-0.158782,-0.266736,-0.468607,-0.17626,-0.326921,-0.313497,-0.309443,-0.218738,-0.218206,-0.253521,-0.249093,-0.030639,0.04139,-0.011425,-0.151082,-0.447164,-0.053367,-0.178785,-0.522276,-0.075699
557.0,0.0,1.3553,0.187724,1.37292,1.367534,0.055384,-0.120693,0.02717,0.017194,0.5196,11.0,7.0,76.0,-0.29855,-0.195138,-0.280807,-0.454103,-0.352025,-0.444179,-0.214674,0.176963

Load in all the train data and drop the `weight` column because it only affects scoring

In [ ]:
# store a linear regression model for each unique symbol_id
linreg_models = {}

for i in range(len(ids)):
    symbol_id = ids[i]
    training_data = train[symbol_id]
    X_train = training_data.select(pl.all().exclude("^responder.*$"))
    X_train.drop_in_place('symbol_id')
    Y_train = training_data['responder_6']
    reg = LinearRegression().fit(X_train, Y_train)
    linreg_models[symbol_id] = reg

In [ ]:
def predict(test, lags):
    global lags_
    global linreg_models
    
    if lags is not None:
        lags_ = lags

    unnecessary_features = ['weight', 'is_scored', 'time_id', 'row_id', 'symbol_id']
    predictions = pl.DataFrame()

    for i in range(len(test)):
        row = test[i]
        symbol_id = row['symbol_id'][0]

        for feature in unnecessary_features:
            row.drop_in_place(feature)

        row = row.fill_nan(0).fill_null(0)

        linreg_model = linreg_models[symbol_id]
        prediction = linreg_model.predict(row)
        prediction = pl.DataFrame(prediction)

        if predictions.is_empty():
            predictions = prediction
        else:
            predictions.extend(prediction)

    predictions = test.with_columns(predictions)
    predictions = predictions.rename({'column_0': 'responder_6'})
    predictions = predictions.select(pl.col(['row_id', 'responder_6']))
    
    if isinstance(predictions, pl.DataFrame):
        assert predictions.columns == ['row_id', 'responder_6']
    elif isinstance(predictions, pd.DataFrame):
        assert (predictions.columns == ['row_id', 'responder_6']).all()
    else:
        raise TypeError('The predict function must return a DataFrame')

    assert len(predictions) == len(test)

    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )